# 5.1. download `production` artifacts

In [ ]:
import os
import pandas as pd
from joblib import load

import wandb
from utils.wandb_utils import get_wandb_artifact

from ml_src.data import preprocess
from ml_src.model import inference

In [ ]:
run = wandb.init(project='ex_census_wandb', job_type='fetch prod model')

In [ ]:
model_artifact, model_path = get_wandb_artifact(run,
                               artifact_name="model", 
                               file_name="model.joblib", 
                               tag="prod", 
                               artifact_type='model_artifact')

encoder_artifact, encoder_path = get_wandb_artifact(run,
                               artifact_name="feature_engineering", 
                               file_name="encoder.joblib", 
                               tag="prod", 
                               artifact_type='feature_engineering_artifact')

lb_artifact, lb_path = get_wandb_artifact(run,
                               artifact_name="feature_engineering", 
                               file_name="lb.joblib", 
                               tag="prod", 
                               artifact_type='feature_engineering_artifact')

cat_features = encoder_artifact.metadata['categorical_feature']
label = encoder_artifact.metadata['label']

In [ ]:
run.finish()

# 5.2. execute inference on new data

In [ ]:
model = load(model_path)
encoder = load(encoder_path)
lb = load(lb_path)

In [ ]:
new_data = {"workclass": "Private",
            "education": "Doctorate",
            "marital-status": "Married-civ-spouse",
            "occupation": "Prof-specialty",
            "relationship": "Husband",
            "race": "White",
            "sex": "Male",
            "native-country": "United-States",
            "hours-per-week": 60}

new_data = pd.DataFrame.from_dict(new_data, orient='index').T

In [ ]:
X_test, _, _, _ = preprocess(X=new_data, 
                             categorical_features=cat_features, 
                             training=False, 
                             encoder=encoder, 
                             lb=lb)

In [ ]:
preds = inference(model, X_test)

In [ ]:
output = lb.inverse_transform(preds)[0]
print(output)